In [15]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import regression as reg
import matplotlib.pyplot as plt

In [16]:
is_terminated = True

In [17]:
def load_data():
    df = pd.read_csv('../data/input/kc_house_data_cleaned.csv')

    X = df[df.columns[range(15)]].to_numpy()
    one = np.ones((X.shape[0], 1))
    X = np.concatenate((one, X), axis=1)
    y = df['price'].to_numpy()
    # y = (y - np.min(y)) / (np.max(y) - np.min(y))
    y = y/1e6
    y = y.reshape((X.shape[0], 1))

    return train_test_split(X, y, test_size=0.2, random_state=42)

In [18]:
X_train, X_test, y_train, y_test = load_data()
print(f'X_train: {X_train.shape}, y_train: {y_train.shape}')
print(f'X_test: {X_test.shape}, y_train: {y_test.shape}')

X_train: (17277, 16), y_train: (17277, 1)
X_test: (4320, 16), y_train: (4320, 1)


In [19]:
import matplotlib.pyplot as plt

def plot_data(lm: reg.RegressionOpt):
    plt.plot(range(len(lm.loss_func_list)), lm.loss_func_list)
    plt.title(f'Loss function', fontsize=16)
    plt.ylabel('Value', fontsize=16)
    plt.xlabel('Count', fontsize=16)
    plt.show()

    plt.plot(range(len(lm.grad_norm_list)), lm.grad_norm_list)
    plt.title(f'Gradient norm', fontsize=16)
    plt.ylabel('Value', fontsize=16)
    plt.xlabel('Count', fontsize=16)
    plt.show()

In [20]:
def save_data(lm: reg.RegressionOpt):
    d = {
        'loss_func_list': lm.loss_func_list,
        'grad_norm_list': lm.grad_norm_list
    }

    df = pd.DataFrame(data=d)

    file_name = 'lf_n_grn_' + str(lm.solver) + '_' + str(lm.step_size) + '.csv'
    df.to_csv('../data/output/' + file_name, index=False)

In [21]:
w_init = np.repeat(0, X_train.shape[1]).reshape((X_train.shape[1], 1))

GD method

In [55]:
import time

start = time.time()

lm = reg.RegressionOpt(
    solver='gd',
    max_iter=10000,
    w=w_init,
    step_size=1,
    X_train=X_train,
    y_train=y_train,
    X_test=X_test,
    y_test=y_test,
    terminate=is_terminated
)

w = lm.fit_gd()

end = time.time()

print(f'gd time: {end - start}')
# plot_data(lm)

# save_data(lm)

# print(f'count: {lm.count}, gradient norm: {lm.grad_norm_list[-1]}, loss func: {lm.loss_func_list[-1]}')
# print(lm.w)

gd time: 6.334485054016113


Newton method

In [52]:
import time

start = time.time()

lm = reg.RegressionOpt(
    solver='newton',
    max_iter=20,
    step_size=1,
    w=w_init,
    X_train=X_train,
    y_train=y_train,
    X_test=X_test,
    y_test=y_test,
    terminate=is_terminated
)

lm.fit_newton()

end = time.time()

print(f'time: {end - start}')

# plot_data(lm)
# save_data(lm)

# print(f'count: {lm.count}, gradient norm: {lm.grad_norm_list[-1]}, loss func: {lm.loss_func_list[-1]}')

time: 0.007977962493896484


Accelerated GD method

In [48]:
import regression as reg

import time
start = time.time()

lm = reg.RegressionOpt(
    solver='agd',
    max_iter=10000,
    step_size=0.5,
    w=w_init,
    X_train=X_train,
    y_train=y_train,
    X_test=X_test,
    y_test=y_test,
    terminate=is_terminated
)

lm.fit_acc_gd()

end = time.time()

print(f'time: {end - start}')

# plot_data(lm)
# save_data(lm)

# print(f'count: {lm.count}, gradient norm: {lm.grad_norm_list[-1]}, loss func: {lm.loss_func_list[-1]}')
# print(lm.w)

time: 0.5395572185516357


In [25]:
# delta = 1e-14
# t0 = 2e-12
# t = []
# for i in range(100):
#     t.append(t0 + i * delta)
#
# for step in t:
#     lm = reg.RegressionOpt(
#     solver='gd',
#     max_iter=500,
#     step_size=step,
#     w=w_init,
#     X_train=X_train,
#     y_train=y_train,
#     X_test=X_test,
#     y_test=y_test,
#     terminate=True
#     )
#
#     lm.fit_gd()
#
#     print(f'step size: {step}, count: {lm.count}, gradient norm: {lm.grad_norm_list[-1]}, loss function: {lm.loss_func_list[-1]}')


GD with backtracking

In [45]:
import regression as reg

import time

start = time.time()

lm = reg.RegressionOpt(
    solver='bgd',
    backtracking=True,
    max_iter=10000,
    step_size=128,
    alpha=0.5,
    beta=0.5,
    w=w_init,
    X_train=X_train,
    y_train=y_train,
    X_test=X_test,
    y_test=y_test,
    terminate=is_terminated
)

lm.fit_gd()

end = time.time()

print(f'gd time: {end - start}')

# plot_data(lm)
# save_data(lm)

# print(f'count: {lm.count}, inner_count: {lm.inner_count}, gradient norm: {lm.grad_norm_list[-1]}, loss func: {lm.loss_func_list[-1]}')
# print(lm.w)

gd time: 1.0643775463104248


Accelerated GD with backtracking

In [27]:
# import regression as reg
#
# t = [4e-13]
# for i in range(25):
#     t.append(1.01 * t[i])
#
# # print(t)
#
# for step in t:
#     lm = reg.RegressionOpt(
#         backtracking=True,
#         max_iter=500,
#         step_size=step,
#         alpha=0.5,
#         beta=0.5,
#         w=w_init,
#         X_train=X_train,
#         y_train=y_train,
#         X_test=X_test,
#         y_test=y_test,
#         terminate=True
#     )
#
#     lm.fit_acc_gd()
#     # plot_data(lm)
#
#     print(f'step size: {step}, count: {lm.count}, gradient norm: {lm.grad_norm_list[-1]}, loss function: {lm.loss_func_list[-1]}')

In [43]:
import time

start = time.time()

lm = reg.RegressionOpt(
    solver='agd_bt',
    backtracking=True,
    max_iter=10000,
    step_size=3,
    alpha=0.5,
    beta=0.5,
    w=w_init,
    X_train=X_train,
    y_train=y_train,
    X_test=X_test,
    y_test=y_test,
    terminate=is_terminated
)

lm.fit_acc_gd()

end = time.time()

print(f'gd time: {end - start}')

# plot_data(lm)
# save_data(lm)

print(f'step size: {lm.step_size}, count: {lm.count}, inner_count: {lm.inner_count}, gradient norm: {lm.grad_norm_list[-1]}, loss function: {lm.loss_func_list[-1]}')

gd time: 0.45752763748168945
step size: 3, count: 161, inner_count: 314, gradient norm: 9.524939212289444e-05, loss function: 0.023215251019049313


In [29]:
# delta = 1e-11
# t0 = 6.251678e-4
# t = []
# for i in range(50):
#     t.append(t0 + i * delta)
# loop = 0
# for step in t:
#     loop += 1
#     lm = reg.RegressionOpt(
#     solver='agd_bt',
#     backtracking=True,
#     max_iter=50,
#     step_size=step,
#     alpha=0.5,
#     beta=0.5,
#     w=w_init,
#     X_train=X_train,
#     y_train=y_train,
#     X_test=X_test,
#     y_test=y_test,
#     terminate=True
#     )
#
#     lm.fit_acc_gd()
#     # plot_data(lm)
#     # save_data(lm)
#
#     print(f'loop: {loop}, step size: {lm.step_size}, count: {lm.count}, inner_count: {lm.inner_count}, gradient norm: {lm.grad_norm_list[-1]}, loss function: {lm.loss_func_list[-1]}')

In [30]:
# w = np.linalg.solve(np.dot(X_train.T, X_train), np.dot(X_train.T, y_train))
# print(w)

In [60]:
from sklearn.linear_model import LinearRegression

import time

start = time.time()
lm = LinearRegression()
lm.fit(X_train, y_train)
end = time.time()

print(f'time: {end - start}')
# lm.coef_
print(f'train score: {lm.score(X_train, y_train)}')
print(f'test score: {lm.score(X_test, y_test)}')

time: 0.017824411392211914
train score: 0.658982195856788
test score: 0.6352020058657151


Accelerated GD with backtracking

In [32]:
# import regression as reg
#
# t = [4e-13]
# for i in range(25):
#     t.append(1.01 * t[i])
#
# # print(t)
#
# for step in t:
#     lm = reg.RegressionOpt(
#         backtracking=True,
#         max_iter=500,
#         step_size=step,
#         alpha=0.5,
#         beta=0.5,
#         w=w_init,
#         X_train=X_train,
#         y_train=y_train,
#         X_test=X_test,
#         y_test=y_test,
#         terminate=True
#     )
#
#     lm.fit_acc_gd()
#     # plot_data(lm)
#
#     print(f'step size: {step}, count: {lm.count}, gradient norm: {lm.grad_norm_list[-1]}, loss function: {lm.loss_func_list[-1]}')

In [33]:
# lm = reg.RegressionOpt(
#     solver='agd_bt',
#     backtracking=True,
#     max_iter=10000,
#     step_size=3,
#     alpha=0.5,
#     beta=0.5,
#     w=w_init,
#     X_train=X_train,
#     y_train=y_train,
#     X_test=X_test,
#     y_test=y_test,
#     terminate=is_terminated
# )

# lm.fit_acc_gd()
# # plot_data(lm)
# save_data(lm)

# print(f'step size: {lm.step_size}, count: {lm.count}, inner_count: {lm.inner_count}, gradient norm: {lm.grad_norm_list[-1]}, loss function: {lm.loss_func_list[-1]}')

In [34]:
# delta = 1e-11
# t0 = 6.251678e-4
# t = []
# for i in range(50):
#     t.append(t0 + i * delta)
# loop = 0
# for step in t:
#     loop += 1
#     lm = reg.RegressionOpt(
#     solver='agd_bt',
#     backtracking=True,
#     max_iter=50,
#     step_size=step,
#     alpha=0.5,
#     beta=0.5,
#     w=w_init,
#     X_train=X_train,
#     y_train=y_train,
#     X_test=X_test,
#     y_test=y_test,
#     terminate=True
#     )
#
#     lm.fit_acc_gd()
#     # plot_data(lm)
#     # save_data(lm)
#
#     print(f'loop: {loop}, step size: {lm.step_size}, count: {lm.count}, inner_count: {lm.inner_count}, gradient norm: {lm.grad_norm_list[-1]}, loss function: {lm.loss_func_list[-1]}')

In [35]:
# w = np.linalg.solve(np.dot(X_train.T, X_train), np.dot(X_train.T, y_train))
# print(w)

In [57]:
from sklearn.linear_model import LinearRegression

lm = LinearRegression()
lm.fit(X_train, y_train)
# lm.coef_
print(f'train score: {lm.score(X_train, y_train)}')
print(f'test score: {lm.score(X_test, y_test)}')

train score: 0.658982195856788
test score: 0.6352020058657151


In [37]:
# import regression as reg
#
# t = [4e-13]
# for i in range(25):
#     t.append(1.01 * t[i])
#
# # print(t)
#
# for step in t:
#     lm = reg.RegressionOpt(
#         backtracking=True,
#         max_iter=500,
#         step_size=step,
#         alpha=0.5,
#         beta=0.5,
#         w=w_init,
#         X_train=X_train,
#         y_train=y_train,
#         X_test=X_test,
#         y_test=y_test,
#         terminate=True
#     )
#
#     lm.fit_acc_gd()
#     # plot_data(lm)
#
#     print(f'step size: {step}, count: {lm.count}, gradient norm: {lm.grad_norm_list[-1]}, loss function: {lm.loss_func_list[-1]}')

In [38]:
# lm = reg.RegressionOpt(
#     solver='agd_bt',
#     backtracking=True,
#     max_iter=50,
#     step_size=6.25158e-4,
#     alpha=0.5,
#     beta=0.5,
#     w=w_init,
#     X_train=X_train,
#     y_train=y_train,
#     X_test=X_test,
#     y_test=y_test,
#     terminate=True
# )

# lm.fit_acc_gd()
# # plot_data(lm)
# # save_data(lm)

# print(f'step size: {lm.step_size}, count: {lm.count}, inner_count: {lm.inner_count}, gradient norm: {lm.grad_norm_list[-1]}, loss function: {lm.loss_func_list[-1]}')

In [39]:
# delta = 1e-11
# t0 = 6.251678e-4
# t = []
# for i in range(50):
#     t.append(t0 + i * delta)
# loop = 0
# for step in t:
#     loop += 1
#     lm = reg.RegressionOpt(
#     solver='agd_bt',
#     backtracking=True,
#     max_iter=50,
#     step_size=step,
#     alpha=0.5,
#     beta=0.5,
#     w=w_init,
#     X_train=X_train,
#     y_train=y_train,
#     X_test=X_test,
#     y_test=y_test,
#     terminate=True
#     )

#     lm.fit_acc_gd()
#     # plot_data(lm)
#     # save_data(lm)

#     print(f'loop: {loop}, step size: {lm.step_size}, count: {lm.count}, inner_count: {lm.inner_count}, gradient norm: {lm.grad_norm_list[-1]}, loss function: {lm.loss_func_list[-1]}')

In [40]:
from sklearn.linear_model import LinearRegression

lm = LinearRegression()
lm.fit(X_train, y_train)
# lm.coef_
print(f'train score: {lm.score(X_train, y_train)}')
print(f'test score: {lm.score(X_test, y_test)}')

train score: 0.658982195856788
test score: 0.6352020058657151
